# Abstract

-- Enter Here --

# Data

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from torchinfo import summary

import pandas as pd
import numpy as np
import time

# for train-test split
from sklearn.model_selection import train_test_split

# for suppressing bugged warnings from torchinfo
import warnings
warnings.filterwarnings("ignore", category = UserWarning)

# tokenizers from HuggingFace
from transformers import BertTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\lukka\anaconda3\envs\ml-0451\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We are loading in a [Kaggle dataset](https://www.kaggle.com/datasets/saurabhshahane/music-dataset-1950-to-2019) that contains information about music made between the years 1950 and 2019 collected through Spotify. The dataset contains lyrics, artist info, track names, etc. Importantly it also includes music metadata like sadness, danceability, loudness, acousticness, etc.

In [2]:
url = "https://raw.githubusercontent.com/PhilChodrow/PIC16B/master/datasets/tcc_ceds_music.csv"
df = pd.read_csv(url)

Lets have a look at some of the raw data!

In [3]:
df.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


Here is a brief look at how many songs we have in each represented genre.

In [4]:
df.groupby("genre").size()

genre
blues      4604
country    5445
hip hop     904
jazz       3845
pop        7042
reggae     2498
rock       4034
dtype: int64

This is a pretty large number of songs to classify... and some genres I personally dont care for. So, to make the dataframe more manageable and applicable to me personally, we are going to narrow down to only observe reggae, hip hop, rock and jazz.

In [5]:
genres = {
    "hip hop"   : 0,
    "jazz" : 1,
    "reggae" : 2,
    "rock" : 3,
}

df = df[df["genre"].apply(lambda x: x in genres.keys())]
df.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
17091,54304,gene ammons,it's the talk of the town,1950,jazz,lovers sweethearts hard understand know happen...,61,0.001096,0.001096,0.001096,...,0.319570,0.001096,0.352323,0.620388,0.868474,0.235830,0.430132,0.282260,sadness,1.0
17092,54305,gene ammons,you go to my head,1950,jazz,head linger like haunt refrain spin round brai...,48,0.001754,0.340964,0.001754,...,0.001754,0.001754,0.379400,0.638541,0.907630,0.900810,0.221970,0.184159,violence,1.0
17093,54307,bud powell,yesterdays,1950,jazz,music speak start hear musicians like dizzy gi...,107,0.001144,0.001144,0.074762,...,0.001144,0.097082,0.489873,0.467400,0.992972,0.927126,0.334295,0.228204,music,1.0
17094,54311,tony bennett,stranger in paradise,1950,jazz,hand stranger paradise lose wonderland strange...,41,0.002105,0.180524,0.002105,...,0.527429,0.002105,0.179032,0.559470,0.983936,0.001781,0.086974,0.235211,sadness,1.0
17095,54313,dean martin,zing-a zing-a zing boom,1950,jazz,zinga zinga zinga zinga zinga zinga zinga zing...,160,0.001253,0.001253,0.001253,...,0.425721,0.001253,0.580851,0.687409,0.655622,0.000000,0.936109,0.418400,sadness,1.0


In [6]:
df["genre"] = df["genre"].apply(genres.get)
df

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
17091,54304,gene ammons,it's the talk of the town,1950,1,lovers sweethearts hard understand know happen...,61,0.001096,0.001096,0.001096,...,0.319570,0.001096,0.352323,0.620388,0.868474,0.235830,0.430132,0.282260,sadness,1.000000
17092,54305,gene ammons,you go to my head,1950,1,head linger like haunt refrain spin round brai...,48,0.001754,0.340964,0.001754,...,0.001754,0.001754,0.379400,0.638541,0.907630,0.900810,0.221970,0.184159,violence,1.000000
17093,54307,bud powell,yesterdays,1950,1,music speak start hear musicians like dizzy gi...,107,0.001144,0.001144,0.074762,...,0.001144,0.097082,0.489873,0.467400,0.992972,0.927126,0.334295,0.228204,music,1.000000
17094,54311,tony bennett,stranger in paradise,1950,1,hand stranger paradise lose wonderland strange...,41,0.002105,0.180524,0.002105,...,0.527429,0.002105,0.179032,0.559470,0.983936,0.001781,0.086974,0.235211,sadness,1.000000
17095,54313,dean martin,zing-a zing-a zing boom,1950,1,zinga zinga zinga zinga zinga zinga zinga zing...,160,0.001253,0.001253,0.001253,...,0.425721,0.001253,0.580851,0.687409,0.655622,0.000000,0.936109,0.418400,sadness,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,0,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,0,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,82449,nine,whutcha want?,2019,0,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,82450,will smith,switch,2019,0,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


The base rate on our classification is the proportion of the data set occupied by the largest label class:

In [7]:
df.groupby("genre").size() / len(df)

genre
0    0.080135
1    0.340839
2    0.221434
3    0.357592
dtype: float64

If we always guessed category 3, then we would expect an accuracy of roughly 36%. So, our task will be to see whether we can train a model to beat this. 

As we try to predict the genre of the track, we will use lyrics alongside some other engineered features (metadata) that we define below.

In [8]:
engineered_features = ['dating', 'violence', 'world/life', 'night/time','shake the audience','family/gospel', 'romantic', 'communication','obscene', 'music', 'movement/places', 'light/visual perceptions','family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability','loudness', 'acousticness', 'instrumentalness', 'valence', 'energy']      

Our models will only need these engineered features, lyrics, and our target value which will be *genre* so we can throw them all into the same dataframe and use slicing to access different parts later.

In [19]:
df_clean= df[engineered_features + ['lyrics', 'genre']].copy()
df_clean.head()

,dating,violence,world/life,night/time,shake the audience,family/gospel,romantic,communication,obscene,music,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,lyrics,genre
17091,0.001096,0.001096,0.001096,0.001096,0.036316,0.001096,0.001096,0.460773,0.086498,0.001096,...,0.319570,0.001096,0.352323,0.620388,0.868474,0.235830,0.430132,0.282260,lovers sweethearts hard understand know happen...,1
17092,0.001754,0.340964,0.001754,0.001754,0.001754,0.001754,0.131872,0.001754,0.001754,0.001754,...,0.001754,0.001754,0.379400,0.638541,0.907630,0.900810,0.221970,0.184159,head linger like haunt refrain spin round brai...,1
17093,0.001144,0.001144,0.074762,0.046173,0.001144,0.018789,0.001144,0.001655,0.001144,0.421734,...,0.001144,0.097082,0.489873,0.467400,0.992972,0.927126,0.334295,0.228204,music speak start hear musicians like dizzy gi...,1
17094,0.002105,0.180524,0.002105,0.002105,0.002105,0.002105,0.002105,0.201965,0.002105,0.002105,...,0.527429,0.002105,0.179032,0.559470,0.983936,0.001781,0.086974,0.235211,hand stranger paradise lose wonderland strange...,1
17095,0.001253,0.001253,0.001253,0.001253,0.001253,0.081126,0.001253,0.111951,0.001253,0.268737,...,0.425721,0.001253,0.580851,0.687409,0.655622,0.000000,0.936109,0.418400,zinga zinga zinga zinga zinga zinga zinga zing...,1


Finally, we will perform a train-validation split to later evaluate our data

In [20]:
df_train, df_val = train_test_split(df_clean,shuffle = True, test_size = 0.2)

# Text Vectorization

We now need to *vectorize* the lyrics. We’re going to use **tokenization** to break up the lyrics into a sequence of tokens, and then vectorize that sequence.

We will be using a tokenizer imported from HuggingFace.

In [9]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

For our purposes it’s more convenient to assign an *integer* to each token, which we can do like this:

In [10]:
encoded = tokenizer("I love reggae music!")
encoded

{'input_ids': [101, 1045, 2293, 15662, 2189, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

To do the reverse, we can use the `.decode` method of the tokenizer:

In [11]:
tokenizer.decode(encoded["input_ids"])

'[CLS] i love reggae music! [SEP]'

Here is some code to help us prepare our dataset with encodings. A lot of our lyrics are different lengths so we will pad the shorter ones with 0s and truncate others that are especially long. We will make use of the torch `Dataset` class to help manage our data.

In [29]:
max_len = 512 # BERT capacity

def pad(l, max_len):
    assert len(l) <= max_len
    to_add = max_len - len(l)
    return l + [0]*to_add

def preprocess(df, tokenizer, max_len):
    # X = tokenizer(list(df["lyrics"]))["input_ids"]
    # X = [pad(t, max_len) for t in X]
    X = tokenizer(list(df["lyrics"]), padding="max_length", truncation=True, max_length=max_len)["input_ids"]
    X = [x + e for x, e in zip(X, df[engineered_features].values.tolist())]
    y = list(df["genre"])
    return X, y

class TextDataFromDF(Dataset):
    def __init__(self, df):
        self.X, self.y = preprocess(df, tokenizer, max_len)

    def __getitem__(self, ix):
        return self.X[ix], self.y[ix]

    def __len__(self):
        return len(self.y)

Lets make our encoded datasets!

In [30]:
train_data = TextDataFromDF(df_train)
val_data   = TextDataFromDF(df_val)

Here is what a single songs information looks like now:

In [35]:
X, y = train_data[1]
print(X)
print(y)

[101, 25237, 25237, 2645, 4682, 2264, 25237, 25237, 2681, 2215, 4536, 8795, 3524, 3524, 3233, 2954, 2655, 2655, 4151, 2455, 2489, 2954, 2204, 2954, 2903, 2157, 25237, 2935, 8391, 2954, 2204, 2954, 25237, 2935, 8391, 2233, 2233, 2455, 2188, 2709, 4828, 10497, 5358, 8739, 7195, 14318, 2015, 2264, 8135, 2272, 10689, 2907, 2152, 15908, 3426, 7354, 27524, 9895, 2368, 2574, 5510, 3886, 4781, 4125, 2892, 2455, 2954, 2204, 2954, 2903, 2157, 25237, 2935, 8391, 2954, 2204, 2954, 25237, 2935, 8391, 2645, 2645, 7354, 27524, 21038, 2015, 3582, 6750, 15834, 4536, 15834, 2954, 3696, 2892, 16579, 2015, 2563, 5000, 8391, 14437, 2668, 5472, 25237, 25237, 5722, 4562, 2925, 6225, 15616, 2954, 2204, 2954, 2903, 2157, 25237, 2935, 8391, 2954, 2204, 2954, 25237, 2935, 8391, 2272, 25237, 2645, 22825, 2954, 2204, 2954, 2903, 2157, 25237, 2935, 8391, 25237, 2935, 8391, 2954, 2204, 2954, 2903, 2157, 25237, 2935, 8391, 25237, 2935, 8391, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

We are going to be feeding data in in batches, so we will need a dataloader which necessitates a collate function to ensure our we are imputing tensors of the right size.

In [36]:
def collate(data):
    X = torch.tensor([d[0] for d in data])
    y = torch.tensor([d[1] for d in data])
    return X,y 

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn = collate)
val_loader = DataLoader(val_data, batch_size=8, shuffle=True, collate_fn = collate)

Here is what a batch of data looks like. The predictor data is now a tensor in which the entries give token indices, padded with 0s and ending with the values of our engineered features. For visualization purposes we’ll show only the first 3 rows:

In [46]:
X, y = next(iter(train_loader))
X[:3]

tensor([[1.0100e+02, 2.1890e+03, 2.3730e+03,  ..., 1.3462e-04, 9.3405e-01,
         5.1550e-01],
        [1.0100e+02, 2.3265e+04, 6.5460e+03,  ..., 2.9757e-03, 8.7428e-01,
         4.4743e-01],
        [1.0100e+02, 3.0420e+03, 4.3760e+03,  ..., 6.2348e-01, 5.2082e-01,
         5.6155e-01]])

In [47]:
y[:3]

tensor([2, 1, 1])

# Model Building 

We are going to train **three** neural networks to classify our genres.

- Using Lyrics to Classify
- Using Engineered Features (Metadata) to Classify
- Using Lyrics and Metadata to Classify

Lets build a model for classifying genres based on lyrics first.

In [53]:
class TextClassificationModel(nn.Module):

    def __init__(self,vocab_size, embedding_dim, max_len, num_class):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size+1, embedding_dim)
        self.fc = nn.Linear(max_len*embedding_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return(x)

In [54]:
vocab_size = len(tokenizer.vocab)
embedding_dim = 10
num_class = len(genres)

text_model = TextClassificationModel(vocab_size, embedding_dim, max_len, num_class).to(device)

In [55]:
summary(text_model, input_Size = (8, max_len))

Layer (type:depth-idx)                   Param #
TextClassificationModel                  --
├─Embedding: 1-1                         305,230
├─Linear: 1-2                            20,484
Total params: 325,714
Trainable params: 325,714
Non-trainable params: 0

In [ ]:
def train(model, dataloader):
    optimizer = torch.optim.Adam(model.parameters(), lr=.1)
    loss_fn = torch.nn.CrossEntropyLoss()

    epoch_start_time = time.time()
    # keep track of some counts for measuring accuracy
    total_acc, total_count = 0, 0
    
    for X, y in dataloader:
        
        # zero gradients
        optimizer.zero_grad()
        # form prediction on batch
        predicted_label = model(X)
        # evaluate loss on prediction
        loss = loss_fn(predicted_label, y)
        # compute gradient
        loss.backward()
        # take an optimization step
        optimizer.step()
                
        # for printing accuracy
        total_acc   += (predicted_label.argmax(1) == y).sum().item()
        total_count += y.size(0)

    print(f'| epoch {epoch:3d} | train accuracy {total_acc/total_count:8.3f} | time: {time.time() - epoch_start_time:5.2f}s')

def accuracy(model, dataloader):

    total_acc, total_count = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            predicted_label = model(X)
            total_acc += (predicted_label.argmax(1) == y).sum().item()
            total_count += y.size(0)
    return total_acc/total_count